In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

In [4]:
from data_analysis import cleaned_data, X, y

ModuleNotFoundError: No module named 'data_analysis'